# analysis

In [6]:
from ast import literal_eval

import pandas as pd
import statsmodels.api as sm

In [39]:
df = pd.read_csv('./data/data.csv').set_index('Steam AppID')

df.tags = df.tags.apply(literal_eval)
df_tags = pd.get_dummies(df[['tags']].explode('tags')).sum(level=0)
df = df.merge(df_tags, how='inner', right_index=True, left_index=True)


/var/folders/75/3hg63jq17tvcpfm66b05wlkc0000gn/T/ipykernel_43029/1220553330.py:4: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df_tags = pd.get_dummies(df[['tags']].explode('tags')).sum(level=0)


In [13]:
# Grab Data
df_scrapped = pd.read_csv('./data/data.csv').drop(['Rating'], axis=1)
df_reviews = pd.read_excel('~/Documents/gdrive/video_games/video_games_master.xlsx')
df = df_reviews.merge(df_scrapped, how='inner', on='Steam AppID').set_index('Title Name')

# Process Tags
df.tags = df.tags.apply(literal_eval)
df['tags'] = df['tags'].apply(lambda row: row[:5])  # Top 5 Tags
df_tags = pd.get_dummies(df[['Steam AppID', 'tags']].explode('tags')).sum(level=0)
df = df.drop(['tags'], axis=1)
df = df.merge(df_tags.drop('Steam AppID', axis=1), how='inner', right_index=True, left_index=True)

df

/var/folders/75/3hg63jq17tvcpfm66b05wlkc0000gn/T/ipykernel_43029/2531933555.py:9: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df_tags = pd.get_dummies(df[['Steam AppID', 'tags']].explode('tags')).sum(level=0)


,System,Owned?,Archived?,Steam AppID,Release Year,Years(s) Played,Omega?,Rating,Review Note,name,...,tags_Tower Defense,tags_Turn-Based,tags_Turn-Based Combat,tags_Turn-Based Strategy,tags_Turn-Based Tactics,tags_Violent,tags_War,tags_Warhammer 40K,tags_World War II,tags_eSports
Title Name,,,,,,,,,,,,,,,,,,,,,
Dark Souls II: Scholar of the First Sin,PC,Steam,NaN,335300.0,2015,2020,Yes,6,"Deliberate, challenging melee combat. Deeply a...",DARK SOULS™ II: Scholar of the First Sin,...,0,0,0,0,0,0,0,0,0,0
Dark Souls III,PC,Steam,NaN,374320.0,2016,2020,Yes,6,"Deliberate, challenging melee combat. Deeply a...",DARK SOULS™ III,...,0,0,0,0,0,0,0,0,0,0
Dark Souls Remastered,PC,Steam,NaN,570940.0,2018,2020,Yes,6,"Deliberate, challenging melee combat. Deeply a...",DARK SOULS™: REMASTERED,...,0,0,0,0,0,0,0,0,0,0
Divinity: Original Sin 2,PC,Steam,NaN,435150.0,2017,2020,Yes,6,Complex and engaging turn-based combat. Flexib...,Divinity: Original Sin 2 - Definitive Edition,...,0,0,0,1,0,0,0,0,0,0
Factorio,PC,Steam,NaN,427520.0,2016,2020,Yes,6,The recursive automaton/machine building loops...,Factorio,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Phasmophobia,PC,Steam,NaN,739630.0,2020,2021,NaN,2,"Can't do horror games, man.",Phasmophobia,...,0,0,0,0,0,0,0,0,0,0
Superbrothers: Sword & Sworcery EP,PC,Steam,NaN,204060.0,2011,2020,NaN,2,the 8-bit graphics made the point-and-click im...,Superbrothers: Sword & Sworcery EP,...,0,0,0,0,0,0,0,0,0,0
War Thunder,PC,NaN,NaN,236390.0,2012,2020,NaN,2,NaN,War Thunder,...,0,0,0,0,0,0,1,0,1,0


In [151]:
# Tags that show up in 10 or more games
tag_cols = list(df[[col for col in df.columns if 'tag' in col]].sum().where(lambda x : x>=5).dropna().index.values)

X = df[['Release Year', 'recent_percent', 'recent_count', 'all_percent', 'all_count'] + tag_cols]
y = df['Rating']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

results.summary()

/Users/mcminnra/miniconda3/envs/sgr/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Rating   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                 -0.046
Method:                 Least Squares   F-statistic:                    0.8626
Date:                Mon, 01 Nov 2021   Prob (F-statistic):              0.653
Time:                        00:09:28   Log-Likelihood:                -116.33
No. Observations:                  83   AIC:                             286.7
Df Residuals:                      56   BIC:                             352.0
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                        -24.8827    115.990     -0.215      0.831    -257.239     207.474
Release Year                   0.0120      0.057      0.208      0.836      -0.103       0.127
recent_percent                 0.0266      0.031      0.863      0.392      -0.035       0.088
recent_count               -2.356e-06   1.05e-05     -0.224      0.824   -2.35e-05    1.88e-05
all_percent                    0.0293      0.036      0.809      0.422      -0.043       0.102
all_count                   9.484e-08   2.68e-07      0.354      0.724   -4.41e-07    6.31e-07
tags_Action                    0.1825      0.426      0.428      0.670      -0.671       1.036
tags_Adventure                 0.1481      0.502      0.295      0.769      -0.857       1.153
tags_Co-op                    -0.2525      0.486     -0.520      0.605      -1.226       0.721
tags_Difficult                 0.2302      0.592      0.389      0.699      -0.955       1.416
tags_Free to Play             -0.1744      0.535     -0.326      0.746      -1.247       0.898
tags_Great Soundtrack         -0.4515      0.672     -0.672      0.504      -1.798       0.895
tags_Indie                     0.0431      0.527      0.082      0.935      -1.012       1.098
tags_Massively Multiplayer    -0.4897      0.741     -0.661      0.512      -1.975       0.995
tags_Multiplayer               0.1493      0.398      0.375      0.709      -0.648       0.947
tags_Online Co-Op             -0.1143      0.494     -0.231      0.818      -1.105       0.876
tags_Open World                0.1002      0.486      0.206      0.837      -0.874       1.074
tags_Pixel Graphics           -0.8125      0.561     -1.448      0.153      -1.936       0.311
tags_RPG                       0.1667      0.440      0.379      0.706      -0.714       1.048
tags_Roguelite                -0.0899      0.687     -0.131      0.896      -1.466       1.287
tags_Singleplayer             -0.5205      0.491     -1.061      0.293      -1.503       0.462
tags_Souls-like                0.5072      0.650      0.781      0.438      -0.794       1.809
tags_Story Rich                0.3096      0.605      0.512      0.611      -0.902       1.521
tags_Strategy                 -0.0553      0.454     -0.122      0.903      -0.965       0.855
tags_Survival                 -0.4416      0.509     -0.868      0.389      -1.461       0.578
tags_Turn-Based                0.0087      0.661      0.013      0.990      -1.315       1.332
tags_Turn-Based Strategy       0.6717      0.635      1.058      0.295      -0.600       1.944
==============================================================================
Omnibus:                        6.934   Durbin-Watson:                   0.801
Prob(Omnibus):                  0.031   Jarque-Bera (JB):                6.

In [149]:
for tag, coef in zip(results.params.sort_values(ascending=False).index, results.params.sort_values(ascending=False)):
    print(f'{tag} => {coef}')

tags_Turn-Based Strategy => 0.6717288293650104
tags_Souls-like => 0.5071876872711089
tags_Story Rich => 0.3096277650409631
tags_Difficult => 0.23022646682217793
tags_Action => 0.18253744494298246
tags_RPG => 0.166654172529757
tags_Multiplayer => 0.14931987870726193
tags_Adventure => 0.1480737394989905
tags_Open World => 0.10016418779975139
tags_Indie => 0.04313267685569755
all_percent => 0.02932849556618648
recent_percent => 0.026621615898386974
Release Year => 0.011952631059206914
tags_Turn-Based => 0.008729273966036073
all_count => 9.484361294564047e-08
recent_count => -2.35616499834874e-06
tags_Strategy => -0.05534390368076858
tags_Roguelite => -0.08989818946164481
tags_Online Co-Op => -0.11433126015863229
tags_Free to Play => -0.17443276953456666
tags_Co-op => -0.25252724735020005
tags_Survival => -0.4415953749905808
tags_Great Soundtrack => -0.4515446089085263
tags_Massively Multiplayer => -0.4896755698988111
tags_Singleplayer => -0.5204742127423645
tags_Pixel Graphics => -0.81248

In [145]:
tag = 'tags_Pixel Graphics'

df[['Rating', tag]][df[tag] == 1]

,Rating,tags_Pixel Graphics
Title Name,,
Into the Breach,5,1
Secrets of Grindea,5,1
Baba is You,4,1
Octopath Traveler,4,1
Hyper Light Drifter,3,1
Kingsway,3,1
Streets of Rogue,3,1
Superbrothers: Sword & Sworcery EP,2,1
